<a href="https://colab.research.google.com/github/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/blob/main/Chatbot-con-FastAPI-Streamlit-y-LangChain/documentos_y_matcomplement/Copia_de_huggingface(a%20ver%20si%20deja%20editar).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aquí monto requeriments2.txt del GitHub pero com la libreria typer==0.09.0, que por un lado debe ser <0.10.00 (por eso he puesto 0.09.0 y por otro mayor que para fastapi-cli 0.0.4 requires typer>=0.12.3, a REVISARLO

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Si se ha instalado longchain y falla, quitar el # del comentario de esta celda

In [ ]:
#!pip uninstall -y langchain langchain-community
!pip install -r /content/drive/MyDrive/Chatbot-con-FastAPI-Streamlit-y-LangChain/requirements/requirements2.txt

  Using cached langchain-0.0.137-py3-none-any.whl (518 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━

SI HACE FALTA WIKI-LANGCHAIN

In [ ]:
%pip install --upgrade --quiet wikipedia langchain langchain_community


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 31.9 MB/s eta 0:00:00


In [ ]:
pip install Wikipedia-API # Faltaba instalar este, añadir luego al requirements

AGENTE DE USUARIO PARA WIKIPEDIA

In [ ]:
import wikipediaapi
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No page found for query: " + query

# Ejemplo de uso
query = "Fibrosis"
result = get_wikipedia_summary(query)

print(result)


Fibrosis, also known as fibrotic scarring, is a pathological wound healing in which connective tissue replaces normal parenchymal tissue to the extent that it goes unchecked, leading to considerable tissue remodelling and the formation of permanent scar tissue.
Repeated injuries, chronic inflammation and repair are susceptible to fibrosis, where an accidental excessive accumulation of extracellular matrix components, such as the collagen, is produced by fibroblasts, leading to the formation of a permanent fibrotic scar.
In response to injury, this is called scarring, and if fibrosis arises from a single cell line, this is called a fibroma. Physiologically, fibrosis acts to deposit connective tissue, which can interfere with or totally inhibit the normal architecture and function of the underlying organ or tissue. Fibrosis can be used to describe the pathological state of excess deposition of fibrous tissue, as well as the process of connective tissue deposition in healing. Defined by t

AHORA INTEGRAMOS LANGCHAIN CON WIKI Y PROBAMOS

In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

PRUEBA DE API KEY DE HUGGING FACE

Aquí o se sube e. .env de GitHub o se meten en el colab en la llave de la izquierda los API KEYS secretas, y se corre la siguiente celda

In [ ]:
from google.colab import userdata
userdata.get('secretName')

SecretNotFoundError: Secret secretName does not exist.

De momento me funciona mejor subiendo el .env de GitHub al root de aquí

In [ ]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv('/content/drive/MyDrive/Chatbot-con-FastAPI-Streamlit-y-LangChain/.env')
add_to_git_credential=True

# Obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if hf_token:
    from huggingface_hub import login
    login(hf_token)
else:
    print("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Obtener otras variables de entorno si es necesario
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key:
    print("GROQ_API_KEY cargado correctamente")
else:
    print("GROQ_API_KEY no encontrado en el archivo .env")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
GROQ_API_KEY cargado correctamente


AHORA PRUEBA DE CODIGO BUENO
Explicación
VectorStoreRetriever: Se utiliza para configurar el recuperador basado en el vectorstore.
load_qa_chain: Carga una cadena de preguntas y respuestas utilizando el PromptTemplate.
RetrievalQA: Se configura con el retriever y combine_documents_chain necesarios.
Pruebo con un pdf pasado a .md de David grabado a saco en el root (/content/sample_data). En PDF funcionaba bien, en MD.
PRUEBA 1

In [ ]:
CARGAR="/content/sample_data/Cholesterol-Myths-vs-Facts-Spanish.md"

In [ ]:
import wikipediaapi
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No page found for query: " + query

# Ejemplo de uso
query = "FIBROSIS"
wikipedia_result = get_wikipedia_summary(query)

print("Wikipedia Result:\n", wikipedia_result)

# Configuración de LangChain y otros módulos
# Cargar documentos
CARGAR = "/content/sample_data/Cholesterol-Myths-vs-Facts-Spanish.md"
# ojo md no pdf quito... pdf_loader = PyPDFLoader(CARGAR)
# ojo si cargo un md ya no es un pdf, por eso cambia el codigo pej...documents =
# pdf_loader.load()
text_loader = TextLoader(CARGAR)
documents = text_loader.load()

# Dividir texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Crear embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Crear el prompt con variables de entrada especificadas
prompt_template = PromptTemplate(
    template="Responde la pregunta basándote en el siguiente contexto: {context}\nPregunta: {question}",
    input_variables=["context", "question"]
)

# Crear la cadena de preguntas y respuestas
qa_chain = load_qa_chain(prompt_template=prompt_template)

# Crear el retrieval QA chain utilizando el vectorstore y la cadena de QA
retrieval_qa = RetrievalQA(
    retriever=vectorstore.as_retriever(),
    combine_documents_chain=qa_chain
)

# Ejemplo de uso de la cadena de preguntas y respuestas
question = "¿Cuál es el tema principal del documento?"
result = retrieval_qa.run(question)

print("QA Result:\n", result)

Wikipedia Result:
 No page found for query: FIBROSIS


NameError: name 'TextLoader' is not defined

PRUEBA 2. ASI CARGARIAMOS LOS PDF DE SU SUBDIRECTORIO

In [ ]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./pdf")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

NameError: name 'PyPDFDirectoryLoader' is not defined

La anterior PRUEBA 1 (de PDF modificado)., lo rehago para cargar .md, y además sigo haciendo pruebas con WIKIPEDIA que fuciona perfecto.

El error normal de longitud indica que el número de tokens en la entrada más los tokens generados exceden el límite de 1024 para el modelo gpt2. Para resolver esto, podemos:

Reducir el tamaño del contexto enviado al modelo.
Utilizar un modelo con un límite de tokens mayor.

In [ ]:
import wikipediaapi
from langchain_community.document_loaders import PyPDFDirectoryLoader  # Asegúrate de que el import es correcto
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()

# Obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if hf_token:
    print("Token de Hugging Face cargado correctamente.")
else:
    print("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No page found for query: " + query

# Ejemplo de uso
query = "Hunter x Hunter"
wikipedia_result = get_wikipedia_summary(query)

print("Wikipedia Result:\n", wikipedia_result)

# Configuración de LangChain y otros módulos
# Cargar documentos
CARGAR = "/content/sample_data/Cholesterol-Myths-vs-Facts-Spanish.md"

# Leer el archivo de texto manualmente
with open(CARGAR, 'r', encoding='utf-8') as file:
    text = file.read()

# Crear una clase de documento simple
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata is not None else {}

# Crear un objeto de documento
documents = [Document(page_content=text)]

# Dividir texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)  # Reducir el tamaño del fragmento
texts = text_splitter.split_documents(documents)

# Crear embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Crear el prompt con variables de entrada especificadas
prompt_template = PromptTemplate(
    template="Responde la pregunta basándote en el siguiente contexto: {context}\nPregunta: {question}",
    input_variables=["context", "question"]
)

# Configurar el modelo de lenguaje de Hugging Face
llm = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.7}, huggingfacehub_api_token=hf_token)

# Crear la cadena de preguntas y respuestas
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Crear el retrieval QA chain utilizando el vectorstore y la cadena de QA
retrieval_qa = RetrievalQA(
    retriever=vectorstore.as_retriever(),
    combine_documents_chain=qa_chain
)

# Ejemplo de uso de la cadena de preguntas y respuestas
question = "¿Cuál es el tema principal del documento?"
result = retrieval_qa(question)

print("QA Result:\n", result)


Token de Hugging Face cargado correctamente.
Wikipedia Result:
 Hunter × Hunter (pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.
Hunter × Hunter was adapted into a 62-episode anime television seri

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


QA Result:
 {'query': '¿Cuál es el tema principal del documento?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nde calorías. Además, tenga en cuenta que la porción en que se basan esos números puede ser menor\n\nque el contenido completo del envase. (Es decir, puede que el envase incluya más de una porción).\n\nMito: Cambiar la mantequilla por margarina me ayudará a bajar el colesterol.\n\nVerdad: No necesariamente. La mantequilla es alta en grasa saturada, pero algunos tipos de\n\nmargarinas tienen incluso un contenido mayor de ambos tipos de grasas. Las margarinas líquidas\n\nfactores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20\n\naños, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años\n\nsiempre y cuando su riesgo se mantenga bajo.\n\nMito: Sólo la gente con sobrepeso 

VOY REVISANDO POR AQUÍ HASTA AHORA CARGA DE MD Y LLM DE HUGGING FACES PERFECTO Y WIKIPEDIA PERFECTA

In [ ]:
len(final_documents)

NameError: name 'final_documents' is not defined

In [ ]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

e:\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568644e-02 -1.19099217e-02 -3.37892585e-02  2.94559337e-02
  5.19159958e-02  5.73839322e-02 -4.10017520e-02  2.74268016e-02
 -1.05128214e-01 -1.58056132e-02  7.94858634e-02  5.64318486e-02
 -1.31765157e-02 -3.41543928e-02  5.81604475e-03  4.72547710e-02
 -1.30746774e-02  3.12988879e-03 -3.44225690e-02  3.08406260e-02
 -4.09086421e-02  3.52737904e-02 -2.43761651e-02 -4.35831733e-02
  2.41503324e-02  1.31986588e-02 -4.84452769e-03  1.92347374e-02
 -5.43912649e-02 -1.42735064e-01  5.15528210e-03  2.93115843e-02
 -5.60810715e-02 -8.53536371e-03  3.14141326e-02  2.76736189e-02
 -2.06188373e-02  8.24231505e-02  4.15425114e-02  5.79654835e-02
 -3.71587239e-02  6.26157876e-03 -2.41390076e-02 -5.61792636e-03
 -2.51715183e-02  5.04967198e-03 -2.52801236e-02 -2.91945064e-03
 -8.24046135e-03 -5.69604561e-02  2.30822619e-02 -5.54219959e-03
  5.11555411e-02  6.09937944e-02  6.49766475e-02 -5.38514033e-02
  2.19109710e-02 -2.54194364e-02 -4.49223518e-02  4.22459245e-02
  4.75252084e-02  7.23217

In [ ]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [ ]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [ ]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000159F4860C10> search_kwargs={'k': 3}


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

e:\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premium to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured.'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr

In [ ]:
a